# AmadeusGPT Demo: Elevated Plus Maze

- please get an openAI user key: https://platform.openai.com/api-keys.
- We suggest to run the demos locally, but it can be viewed on Google Colab. Some interactive features might not be available.

In [ ]:
!pip install --pre amadeusgpt

In [ ]:
import amadeusgpt

- Let's test that your open AI API Key works:

In [ ]:
mykey = "paste-your-key-here"

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=mykey)

response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Hello"}]
)
print(response.choices[0].message.content)

In [ ]:
#If th openai api key is not set already, please set it here.
import os
if 'OPENAI_API_KEY' not in os.environ:
     os.environ['OPENAI_API_KEY'] = mykey
amadeusgpt.__file__

In [ ]:
%matplotlib widget
from amadeusgpt import AMADEUS
import amadeusgpt
from pathlib import Path
import amadeusgpt
from amadeusgpt.utils import parse_result
from amadeusgpt import create_project

## Please upload the demo video and associated files:
- you can grab it from here: https://github.com/AdaptiveMotorControlLab/AmadeusGPT/tree/mwm/docs/examples/EPM


In [ ]:
from google.colab import files

uploaded = files.upload()
for filepath, content in uploaded.items():
  print(f'User uploaded file "{filepath}" with length {len(content)} bytes')

video_path = Path(filepath).resolve()

- Set the scene number to visualize your video in a specific frame

- 🔥 Make sure your animal(s) are visible on that frame so gpt-4o can configure AmadeusGPT correctly

In [ ]:
scene_frame_number = 400
amadeus_root = Path(amadeusgpt.__file__).parent.parent

kwargs = {
    "keypoint_info.body_orientation_keypoints.neck" : "nose",
    "keypoint_info.body_orientation_keypoints.tail_base" : "tail_base",
    "keypoint_info.body_orientation_keypoints.animal_center" : "neck",
    "keypoint_info.head_orientation_keypoints.nose" : "nose",
    "keypoint_info.head_orientation_keypoints.neck" : "neck",
    "video_info.scene_frame_number" : scene_frame_number,
}

config = create_project(data_folder = "../content", # if you use locally: "../examples/EPM"
                        result_folder = "EPM_results",
                        **kwargs
                        )
amadeus = AMADEUS(config, use_vlm=True)
video_file_paths = amadeus.get_video_file_paths()
print (video_file_paths)

#### 🚨 warning, if you see an error `AttributeError: 'NoneType' object has no attribute 'choices'`, look above for openAI errors

## Draw ROIs. Press Esc when you are done drawing each ROI.
- After you are done just run the next cell!

In [ ]:
behavior_analysis = amadeus.get_behavior_analysis('../content/EPM_11.mp4') #check the path!
behavior_analysis.gui_manager.add_roi_from_video_selection()

### Get video clips, ethogram and trajectory plots for mouse in the ROI 0

In [ ]:
query = "When is the mouse in ROI0"
qa_message = amadeus.step(query)
qa_message = parse_result(amadeus, qa_message)

### You can get a list of binary masks (equivalent to ethogram) for the underlying behavior, if your query is about retriving a described behavior

In [ ]:
# the return masks is of shape (num_of_events, video_length)
# where each boolean array of (video_length,) is binary where True indicates whether the behavior is happening at that frame
masks = qa_message.get_masks()
print (masks)

In [ ]:
query = "Plot the trajectory of the animal using the animal center and color it by time"
qa_message = amadeus.step(query)
qa_message = parse_result(amadeus, qa_message)

### How to retrieve results using the query

In [ ]:
messages = amadeus.get_messages()

for query, qa_message in messages.items():
    print (query)
    print (qa_message.get_masks())